In [1]:
import os
import numpy as np
import SimpleITK as sitk

In [18]:
import nibabel as nib
sample='IXI012-HH-1211-T2.nii.gz'
img = nib.load(os.path.join(path,sample))  # 或 'your_file.nii.gz'
data = img.get_fdata()
#print(type(data))
# 计算值范围
min_val = data.min()
max_val = data.max()
y=1334
total_pixels = data.size
pixels_leq_y = np.sum(data <= y)  # 统计≤y的像素数量
percentile = (pixels_leq_y / total_pixels) * 100
print(max_val)
print(percentile)

2025.6734615564346
99.93092673165458


In [19]:
path='D:\\data\\IXI-T2'
files=os.listdir(path)
sample='IXI423-IOP-0974-T2.nii.gz'

In [ ]:
def auto_window_level(image, lower_percentile=1, upper_percentile=99.93):
    """
    ITK-SNAP Auto Contrast的Python实现
    :param image: SimpleITK图像或NumPy数组
    :param lower_percentile: 下限百分位（默认1%）
    :param upper_percentile: 上限百分位（默认99%）
    :return: (window_width, window_level)
    """
    array = sitk.GetArrayFromImage(image) if isinstance(image, sitk.Image) else image
    low = np.percentile(array, lower_percentile)  # 计算下限值（类似ITK-SNAP的自动最小值）
    high = np.percentile(array, upper_percentile) # 计算上限值（类似ITK-SNAP的自动最大值）
    min_val = array.min()
    max_val = array.max()
    window_width = high - low                    # 窗宽 = 最大值 - 最小值
    window_level = (high + low) / 2             # 窗位 = (最大值 + 最小值) / 2
    #print(max_val,high)
    return window_width, window_level,min_val,max_val

def apply_window_level(image, window_width, window_level,min,max):
    """将窗宽窗位应用到图像（类似ITK-SNAP的线性对比度调整）"""
    array = sitk.GetArrayFromImage(image)
    min_val = window_level - window_width / 2
    max_val = window_level + window_width / 2
    adjusted = np.clip((array - min_val) / (max_val - min_val) * max, 0, max).astype(np.float32)
    return adjusted

In [30]:
# 读取NIfTI图像
dirpath='D:\\data\\IXI-T2-lca'
os.makedirs(dirpath,exist_ok=True)
for file in files:
    image = sitk.ReadImage(os.path.join(path,file))

    # 1. 计算自动窗宽窗位（等效ITK-SNAP Auto）
    ww, wl,minv,maxv = auto_window_level(image)  
    #print(ww,wl)
    # 2. 应用调整（等效Linear Contrast Adjustment）
    adjusted_array = apply_window_level(image, ww, wl,minv,maxv)

    # 3. 保存结果
    adjusted_image = sitk.GetImageFromArray(adjusted_array)
    adjusted_image.CopyInformation(image)  # 保留原始图像的元数据
    sitk.WriteImage(adjusted_image, os.path.join(dirpath,file))

775 584.0
2025.6735 1333.9423828125
2634.4802 1258.6588134765625
2512.691 1636.0439453125
2630.9978 1317.087646484375
2130.859 1808.9305419921875
2274.392 1795.927490234375
2145.1812 1812.005859375
2191.5684 1758.666015625
2128.73 1798.4615478515625
2208.296 1840.846923828125
2154.4934 1751.2071533203125
2198.2334 1870.233154296875
821 589.0
2164.899 1792.9559326171875
2297.708 1716.933837890625
658 559.0
551 446.0
2357.5706 1772.7706298828125
706 558.0
2073.776 1241.8935546875
1793.3267 1337.145751953125
3634 2544.0
2210.9436 1732.3175048828125
2184.0244 1828.9713134765625
2205.0837 1801.098876953125
2082.3113 1323.1937255859375
2189.4602 1792.9903564453125
2190.3484 1852.3907470703125
2167.1255 1813.30908203125
2145.5894 1805.3985595703125
2131.5552 1823.081298828125
2928.477 1701.8084716796875
700 587.0
1709.4735 1256.2154541015625
1741.7391 1365.308349609375
2264.1 1853.543212890625
1997.1355 1420.7264404296875
1708.3833 1210.98681640625
2272.0 1836.800048828125
2149.6033 1789.3511